In [1]:
from collections import defaultdict
import re


def tokenize(message):
    """ Removes all punctuation and returns a simple set
    of all the words in the given string (pushed to lower case)
    """
    message = message.lower()
    all_words = re.findall("[a-z0-9]+", message)
    return set(all_words)


def count_words(training_set):
    """ training set consists of pairs (message, is_spam) """
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts


In [2]:
# Here is a quick test of tokenize

mess = """Hey Emmy,
How are you doing? Are things good?
Things are good? Ah, good. Glad to hear it.
Listen if you can loren ipsum, blah blah blah...
-John"""

print(tokenize(mess))

{'ah', 'glad', 'listen', 'good', 'if', 'it', 'loren', 'blah', 'hey', 'how', 'emmy', 'can', 'are', 'john', 'doing', 'to', 'things', 'ipsum', 'you', 'hear'}


In [3]:
import math


def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """ turn the word counts into a list of triples:
    w, p(w | spam) and p(w | ~spam)
    """
    return [(w,
            (spam + k) / (total_spams + 2 * k),
            (non_spam + k) / (total_non_spams + 2 * k))
            for w, (spam, non_spam) in counts.items()]


def log(real):
    try:
        return math.log(real)
    except:
        return 0.0


def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    # iterate through each word in our vocab
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        if word in message_words:
            # if "word" in message, add the log prob of seeing it
            log_prob_if_spam += log(prob_if_spam)
            log_prob_if_not_spam += log(prob_if_not_spam)
        else:
            # if "word" not in message, ad log prob of Not seeing it
            log_prob_if_spam += log(1.0 - prob_if_spam)
            log_prob_if_not_spam += log(1.0 - prob_if_not_spam)
        
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)

    return prob_if_spam / (prob_if_spam + prob_if_not_spam)


class NaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
    
    def train(self, training_set):
        """train the classifier"""
        # count spam and non-spam messages
        num_spams = len([is_spam for message, is_spam in training_set if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)
    
    def classify(self, message):
        return spam_probability(self.word_probs, message)



In [4]:
""" We want a lot of test spam emails. The good folks at Apache have us covered:
https://spamassassin.apache.org/publiccorpus/
"""
import glob
import re

path = 'emails/*/*'
data = []

# glob.glob returns file names in a path, allowing for wild cards
for file_path in glob.glob(path):
    is_spam = 'ham' not in file_path

    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            for line in f:
                if line.startswith('Subject:'):
                    subject = re.sub(r"^Subject: ", "", line).strip()
                    data.append((subject, is_spam))
        except:
            # some of these files have bizarre encodings, ignore them
            pass

# TODO: Learn to deal with UTF-8 text files in Python v3:
#       http://stackoverflow.com/questions/11918512/python-unicodedecodeerror-utf8-codec-cant-decode-byte

print(len(data))

4013


In [5]:
""" Now let's split our data up into training data and test data.
Let's go for a 3:1 ratio.
"""
from random import random


def split_data(data, prob):
    """split data into fractions [prob, 1 -prob]"""
    results = [], []
    for row in data:
        results[0 if random() < prob else 1].append(row)
    return results


train_data, test_data = split_data(data, 0.75)
classifier = NaiveBayesClassifier()
classifier.train(train_data)

In [6]:
""" Now let's see how well are model is doing. """
from collections import Counter

# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject)) for subject, is_spam in test_data]

# assume that spam_probability > 0.5 corresponds to correct prediction (this is a bit iffy)
counts50 = Counter((is_spam, spam_probability > 0.5) for _, is_spam, spam_probability in classified)

print('\nAssume >50% means spam:')
print(counts50)
precision50 = counts50[(True, True)] / (counts50[(True, True)] + counts50[(True, False)])
recall50 = counts50[(True, True)] / (counts50[(True, True)] + counts50[(False, False)])
accuracy50 = (counts50[(True, True)] + counts50[(False, False)]) / (sum(counts50.values()))
f1_50 = 2 * precision50 * recall50 / (precision50 + recall50)
print('Precision: ' + '%.2f' % precision50)
print('Recall: ' + '%.2f' % recall50)
print('Accuracy: ' + '%.2f' % accuracy50)
print('F1 Score: ' + '%.2f' % f1_50)

# assume that spam_probability > 0.75 corresponds to correct prediction (this is a bit iffy)
counts75 = Counter((is_spam, spam_probability > 0.75) for _, is_spam, spam_probability in classified)

print('\nAssume >75% means spam:')
print(counts75)
precision75 = counts75[(True, True)] / (counts75[(True, True)] + counts75[(True, False)])
recall75 = counts75[(True, True)] / (counts75[(True, True)] + counts75[(False, False)])
accuracy75 = (counts75[(True, True)] + counts75[(False, False)]) / (sum(counts75.values()))
f1_75 = 2 * precision75 * recall75 / (precision75 + recall75)
print('Precision: ' + '%.2f' % precision75)
print('Recall: ' + '%.2f' % recall75)
print('Accuracy: ' + '%.2f' % accuracy75)
print('F1 Score: ' + '%.2f' % f1_75)

# assume that spam_probability > 0.95 corresponds to correct prediction (this is a bit iffy)
counts95 = Counter((is_spam, spam_probability > 0.95) for _, is_spam, spam_probability in classified)

print('\nAssume >95% means spam:')
print(counts95)
precision95 = counts95[(True, True)] / (counts95[(True, True)] + counts95[(True, False)])
recall95 = counts95[(True, True)] / (counts95[(True, True)] + counts95[(False, False)])
accuracy95 = (counts95[(True, True)] + counts95[(False, False)]) / (sum(counts95.values()))
f1_95 = 2 * precision95 * recall95 / (precision95 + recall95)
print('Precision: ' + '%.2f' % precision95)
print('Recall: ' + '%.2f' % recall95)
print('Accuracy: ' + '%.2f' % accuracy95)
print('F1 Score: ' + '%.2f' % f1_95)


Assume >50% means spam:
Counter({(False, False): 628, (True, True): 290, (False, True): 67, (True, False): 38})
Precision: 0.88
Recall: 0.32
Accuracy: 0.90
F1 Score: 0.47

Assume >75% means spam:
Counter({(False, False): 661, (True, True): 264, (True, False): 64, (False, True): 34})
Precision: 0.80
Recall: 0.29
Accuracy: 0.90
F1 Score: 0.42

Assume >95% means spam:
Counter({(False, False): 683, (True, True): 221, (True, False): 107, (False, True): 12})
Precision: 0.67
Recall: 0.24
Accuracy: 0.88
F1 Score: 0.36


In [7]:
""" So, what do we learn from the above test?
Well, it's impressive that our accuracy is at 88%, after only parsing the
subjects of the email. But our F1 scores are less impressive.
Let's take a look at the most misclassified."""
# sort by spam probability from smallest to largest
classified.sort(key=lambda row: row[2])

# the highest predicted spam probs among the non-spams
spammiest_hams = list(filter(lambda row: not row[1], classified))[-5:]

# the lowest predicted spam probs among the actual hams
hammiest_spams = list(filter(lambda row: row[1], classified))[:5]

print('\nspammiest hams')
print(spammiest_hams)

print('\nhammiest spams')
print(hammiest_spams)


spammiest hams
[('100 not safe for work pics for chicks. WEENERS.', False, 0.9886800829267236), ("Month by month, 'Most Beautiful Man' winners and their galleries. sfw", False, 0.9915721334814532), ('Ray Ozzie: "How long before we see auto pingback generator spambots?"', False, 0.9976091371596136), ("Right-wing governments 'increase suicide rates'", False, 0.9979303630094196), ('=?iso-8859-1?Q?Matrox_Parhelia=99_now_available?=', False, 0.9996298713932731)]

hammiest spams
[('RE: Own An Automated Shopping Mall                       32736', True, 0.0002947047466527441), ('Re: Funny', True, 0.002334867109411576), ('WOMEN WITH CUM ON THEIR FACE!!!', True, 0.0148153543885132), ('RE: Surprise :-)', True, 0.014817930066454417), ('Fwd: next Tuesday at 9am', True, 0.016280551300562402)]


In [8]:
""" Okay, now let's try training on the actual bodies of all these emails.
This is an experiment.
"""

def find_tags(file_path):
    """Parse an email to get a set of header tags."""
    tags = set()
    f = open(file_path, 'r')
    for line in f.readlines():
        if line.strip() == '': break
        if line[0] in ['>', '<']: break
        if ':' in line[:32] and ' ' not in line[:32].split(':')[0]:
            tags.add(line.split(':')[0])
    
    f.close()
    return tags


tags = set()
path = 'emails/*/*'
files = glob.glob(path)

files = glob.glob(path)
for file_path in files:
    try:
        tags.update(find_tags(file_path))
    except:
        # There are UTF-8 non-compliant emails, I'm just ignoring them for now.
        pass

print(len(tags))

352


In [42]:
def tokenize_email_line(message):
    all_words = re.findall("[a-z]+", message.lower())
    return set(all_words)


def tokenize_email(body, tags):
    text = set()
    header = True
    for line in body.split('\n')[1:]:
        if header:
            if line[0] in [' ', '\t']:
                continue
            elif ':' not in line:
                header = False
            elif line.split(':')[0] in tags:
                continue
        else:
            text.update(tokenize_email_line(line))

    return set(text)


def count_words_email(training_set):
    """ training set consists of pairs (message, is_spam) """
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize_email(message, tags):
            counts[word][0 if is_spam else 1] += 1
    return counts


def spam_probability_email(word_probs, message):
    message_words = tokenize_email(message, tags)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    # iterate through each word in our vocab
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        if word in message_words:
            # if "word" in message, add the log prob of seeing it
            log_prob_if_spam += log(prob_if_spam)
            log_prob_if_not_spam += log(prob_if_not_spam)
        else:
            # if "word" not in message, ad log prob of Not seeing it
            log_prob_if_spam += log(1.0 - prob_if_spam)
            log_prob_if_not_spam += log(1.0 - prob_if_not_spam)
        
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)

    return prob_if_spam / (prob_if_spam + prob_if_not_spam)


class NaiveBayesClassifierEmail(NaiveBayesClassifier):
    
    def __init__(self, k=0.5):
        NaiveBayesClassifier.__init__(self, k)
    
    def train(self, training_set):
        """train the classifier"""
        # count spam and non-spam messages
        num_spams = len([is_spam for message, is_spam in training_set if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        # run training data through our "pipeline"
        word_counts = count_words_email(training_set)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)
    
    def classify(self, message):
        return spam_probability_email(self.word_probs, message)


In [43]:
"""Time to train on the bodies of the emails."""

path = 'emails/*/*'
data2 = []

# glob.glob returns file names in a path, allowing for wild cards
for file_path in glob.glob(path):
    is_spam = 'ham' not in file_path
    
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            text = set()
            _ = f.readline()
            header = True
            for line in f.readlines():
                if header:
                    if line[0] in [' ', '\t']:
                        continue
                    elif ':' not in line:
                        header = False
                    elif line.split(':')[0] in tags:
                        continue
                else:
                    for word in tokenize_email_line(line):
                        data2.append((word, is_spam))
        except:
            # some of these files have bizarre encodings, ignore them
            pass

print(len(data2))


train_data2, test_data2 = split_data(data2, 0.75)
classifier2 = NaiveBayesClassifierEmail()
classifier2.train(train_data2)

1753647


In [44]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier2.classify(subject)) for subject, is_spam in test_data2]

# assume that spam_probability > 0.5 corresponds to correct prediction (this is a bit iffy)
counts50 = Counter((is_spam, spam_probability > 0.5) for _, is_spam, spam_probability in classified)

print('\nAssume >50% means spam:')
print(counts50)
precision50 = counts50[(True, True)] / (counts50[(True, True)] + counts50[(True, False)])
recall50 = counts50[(True, True)] / (counts50[(True, True)] + counts50[(False, False)])
accuracy50 = (counts50[(True, True)] + counts50[(False, False)]) / (sum(counts50.values()))
#f1_50 = 2 * precision50 * recall50 / (precision50 + recall50)
print('Precision: ' + '%.2f' % precision50)
print('Recall: ' + '%.2f' % recall50)
print('Accuracy: ' + '%.2f' % accuracy50)
#print('F1 Score: ' + '%.2f' % f1_50)



Assume >50% means spam:
Counter({(False, False): 254584, (True, False): 184465})
Precision: 0.00
Recall: 0.00
Accuracy: 0.58


In [38]:
print(len(test_data2))
print(len([t for t in test_data2 if t[1]]))

457908
186551
